In [ ]:
from manim import *
import random

class FitLineAndDuplicate(Scene):
    def construct(self):
        # ---------- 坐标系（第一象限） ----------
        axes = Axes(
            x_range=[0, 6, 1],
            y_range=[0, 6, 1],
            x_length=6,
            y_length=6,
            tips=True
        ).to_edge(ORIGIN)
        self.add(axes)

        # ---------- 生成噪声数据点（左边坐标系） ----------
        points = []
        x_values = [0.5, 1, 1.5, 2, 3, 3.5, 4, 5]
        for x in x_values:
            y = 0.5 * x + 1 + random.uniform(-0.6, 0.6)
            dot = Dot(axes.coords_to_point(x, y), color=RED)
            points.append(dot)
        points_group = VGroup(*points)
        self.play(FadeIn(points_group))
        self.wait(1)

        # ---------- 拟合直线 ----------
        line = axes.plot(lambda x: 0.5*x + 1, color=BLUE_C)
        self.play(Create(line))
        self.wait(1)

        # ---------- 红色垂线示意 ----------
        x_red = 4.6  # x轴红点位置
        y_red = 0
        red_dot = Dot(axes.coords_to_point(x_red, y_red), color=RED).scale(1.4)
        self.play(FadeIn(red_dot))
        self.wait(0.5)

        # 红色虚线从x轴红点到蓝线
        y_intersect = 0.5 * x_red + 1  # 与蓝线交点的y坐标
        vertical_line = DashedLine(
            start=axes.coords_to_point(x_red, 0),
            end=axes.coords_to_point(x_red, y_intersect),
            color=RED
        )
        self.play(Create(vertical_line))
        self.wait(0.5)

        # 交点金色点
        gold_dot = Dot(axes.coords_to_point(x_red, y_intersect), color=GOLD).scale(2)
        self.play(FadeIn(gold_dot))
        self.wait(1)

        # ---------- 整个坐标系平移到左边 ----------
        self.play(
            axes.animate.shift(LEFT*3.5),
            points_group.animate.shift(LEFT*3.5),
            line.animate.shift(LEFT*3.5),
            red_dot.animate.shift(LEFT*3.5),
            vertical_line.animate.shift(LEFT*3.5),
            gold_dot.animate.shift(LEFT*3.5)
        )
        self.wait(0.5)

        # ---------- 右边新坐标系 ----------
        axes2 = axes.copy().shift(RIGHT*7)
        self.add(axes2)

        # ---------- 右边数据点：两个簇 ----------
        cluster1_center = (1.5, 1.5)
        cluster2_center = (3.5, 3.5)
        cluster1 = VGroup(*[
            Dot(
                axes2.coords_to_point(
                    cluster1_center[0] + random.uniform(-0.7, 0.4),
                    cluster1_center[1] + random.uniform(-0.4, 0.7)
                ),
                color=BLUE_C
            ) for _ in range(5)
        ])
        cluster2 = VGroup(*[
            Dot(
                axes2.coords_to_point(
                    cluster2_center[0] + random.uniform(-0.6, 0.6),
                    cluster2_center[1] + random.uniform(-0.7, 0.6)
                ),
                color=RED_C
            ) for _ in range(5)
        ])
        self.play(FadeIn(cluster1), FadeIn(cluster2))
        self.wait(2)

        # ---------- 两个坐标系同时移动 ----------
        left_group = VGroup(axes, points_group, line, red_dot, vertical_line, gold_dot)
        right_group = VGroup(axes2, cluster1, cluster2)

        self.play(
            left_group.animate.shift(LEFT*7),   # 左边整体移出屏幕
            right_group.animate.shift(LEFT*3.5),  # 右边移到屏幕中央
            run_time=2
        )
        self.wait(2)
        # ---------- 添加分隔直线 ----------
        separator = axes2.plot(lambda x: -x + 5.5, color=GOLD_D, stroke_width=3)
        self.play(Create(separator))
        self.wait(2)
        # ---------- 添加公式 ----------
        formula1 = MathTex("z = wx + b").next_to(axes2, UP).scale(1.5).shift(DOWN*0.1)
        self.play(Write(formula1))
        self.wait(2)
        # 替换为 z = w^T x + b
        formula2 = MathTex("z = w^T x^T + b").move_to(formula1).scale(1.5).shift(DOWN*0.1)
        self.play(TransformMatchingTex(formula1, formula2))
        self.wait(2)
        ff = Text("得分函数", font="Microsoft YaHei", font_size=45, color=WHITE)
        ff.next_to(formula1, LEFT, buff=1.5).shift(DOWN*0.15)
        self.play(Write(ff))
        self.wait(2)     

        # ---------- 直线函数 ----------
        def separator_func(x):
            return -x + 5.5
        # 获取坐标系边界
        x_min, x_max = axes2.x_range[0], axes2.x_range[1]-0.5
        y_min, y_max = axes2.y_range[0], axes2.y_range[1]-0.5
        # 找到直线在边界的两个交点
        p1 = axes2.coords_to_point(x_min, separator_func(x_min))  # 左边界交点
        p2 = axes2.coords_to_point(x_max, separator_func(x_max))  # 右边界交点
        # 左下区域（蓝色半透明）
        region_blue = Polygon(
            axes2.coords_to_point(x_min, y_min),  # 左下角
            axes2.coords_to_point(x_max, y_min),  # 右下角
            p2,                                   # 直线右交点
            p1,                                   # 直线左交点
            color=BLUE, fill_opacity=0.3, stroke_opacity=0
        )
        # 右上区域（红色半透明）
        region_red = Polygon(
            p1,
            p2,
            axes2.coords_to_point(x_max, y_max),  # 右上角
            axes2.coords_to_point(x_min, y_max),  # 左上角
            color=RED, fill_opacity=0.3, stroke_opacity=0
        )
        # 添加到场景
        self.play(FadeIn(region_blue), FadeIn(region_red))
        self.wait(5)
        
        # 在公式右边加一行：x ∈ (-∞, +∞)
        domain = MathTex("\\in (-\\infty, +\\infty)").scale(1.2)
        domain.next_to(formula2, RIGHT, buff=0.8).shift(DOWN*0.05)
        self.play(Write(domain))
        self.wait(2)
        self.play(domain.animate.set_color(RED), run_time=1.5)
        self.wait(1.5)

        gray_screen = Rectangle(
            width=self.camera.frame_width,
            height=self.camera.frame_height,
            fill_color=DARK_GRAY,
            fill_opacity=0.9,
            stroke_width=0,
            z_index=0
        )
        self.play(FadeIn(gray_screen))
        # ---------- 表格数据 ----------

        table = Table(
            [["138"], ["146"], ["140"]],  # 分数列
            row_labels=[
                Text("语文", font="Microsoft YaHei", color=WHITE),
                Text("数学", font="Microsoft YaHei", color=WHITE),
                Text("外语", font="Microsoft YaHei", color=WHITE),
            ],
            col_labels=[Text("分数", font="Microsoft YaHei", color=WHITE)],
            include_outer_lines=True,
            element_to_mobject=lambda el: Text(el, font="Microsoft YaHei", color=WHITE),
        )
        table.scale(0.75)
        table.move_to(ORIGIN)

        # 表格淡入
        self.play(FadeIn(table))
        self.wait(2)
        # 区间文字 [0,150]（最终位置在表格右边）
        interval = MathTex("\\in [ 0,150 ]").scale(1.6)
        interval.next_to(table, RIGHT, buff=1.5).shift(UP*1.8+LEFT*3.8)
        # 初始：稍微靠右 & 透明
        interval.shift(RIGHT*0.8)
        interval.set_opacity(0)
        self.add(interval)
        # 表格左移 + 区间逐渐显现并滑入
        self.play(
            table.animate.shift(LEFT*3),
            interval.animate.shift(LEFT*0.8).set_opacity(1),
            run_time=2
        )
        self.wait(2)
        # ---------- 150 -> 1500 动画 ----------
        interval2 = MathTex("\\in [ 0,1500 ]").scale(1.6).set_color(RED)
        interval2.move_to(interval)  # 保持位置一致
        self.play(TransformMatchingTex(interval, interval2))
        self.wait(2)
        # ---------- 新表格（2行2列） ----------
        new_table = Table(
            [["1.0"], ["4.0"]],
            row_labels=[Text("A", font="Microsoft YaHei", color=WHITE),
                        Text("B", font="Microsoft YaHei", color=WHITE)],
            col_labels=[Text("绩点", font="Microsoft YaHei", color=WHITE)],
            include_outer_lines=True,
            h_buff=1.5,  # 列间距
            v_buff=1.0,  # 行间距
            element_to_mobject=lambda el: Text(el, font="Microsoft YaHei", color=WHITE),
        )
        new_table.scale(0.9)
        new_table.move_to(RIGHT*8)  # 初始屏幕右边

        # ---------- 左移旧表格 + 区间，右移新表格 ----------
        self.play(
            table.animate.shift(LEFT*8),
            interval2.animate.shift(LEFT*10),
            new_table.animate.move_to(ORIGIN),
            run_time=2
        )
        self.wait(2)
        ii = MathTex("\\in [ 0.0,4.0 ]").scale(1.6)
        ii.next_to(new_table, RIGHT, buff=1.5).shift(UP*1.55+LEFT*3.8)
        ii.shift(RIGHT*0.8)
        ii.set_opacity(0)
        self.add(ii)
        self.play(
            new_table.animate.shift(LEFT*3.0),
            ii.animate.shift(LEFT*0.8).set_opacity(1),
            run_time=2
        )
        self.wait(2)
        ii2 = MathTex("\\in [ 0.0,100.0 ]").scale(1.6).set_color(RED)
        ii2.move_to(ii)
        self.play(TransformMatchingTex(ii, ii2))
        self.wait(5)
        # 最终元素打包
        final_group = VGroup(new_table, ii2)
        # 左移 + 灰色淡出
        self.play(
            final_group.animate.shift(LEFT*11),
            gray_screen.animate.set_opacity(0),  # 灰色淡出
            run_time=2
        )
        self.wait(3)

        # ---------- 初始化文字 ----------
        magic_text = Text("Magic", font="Arial", font_size=100, color=YELLOW).move_to(ORIGIN)
        activation_text = Text("激活函数", font="Microsoft YaHei", font_size=100, color=GREEN).move_to(ORIGIN)
        activation_text.set_opacity(0)  # 初始透明

        # 将之前的所有元素打包为 old_group
        old_group = VGroup(
            axes, points_group, line, red_dot, vertical_line, gold_dot,
            axes2, cluster1, cluster2, separator, formula2, ff,
            region_blue, region_red, domain, table, interval2, new_table, ii2, gray_screen
        )
        self.add(magic_text, activation_text, old_group)
        self.wait(1)
        # ---------- 参数 ----------
        magic_rotations = 5        # Magic旋转圈数
        magic_duration = 5         # Magic旋转总时长
        steps = 150                # 分帧数
        # ---------- 非线性速度函数：ease-in-out ----------
        def speed_curve(t):
            return np.sin(t * np.pi)  # 开慢中间快末慢
        # ---------- Magic旋转 + 渐变 ----------
        for i in range(steps):
            t = i / steps
            angle_step = 2 * PI * magic_rotations / steps * speed_curve(t) * 1.5
            # 渐变替换文字：在最后20%时开始出现激活函数
            if t >= 0.8:
                alpha = (t - 0.8) / 0.2
                activation_text.set_opacity(alpha)
                magic_text.set_opacity(1 - alpha)
            magic_text.rotate(angle_step, axis=UP)
            activation_text.rotate(angle_step, axis=UP)

            old_group.set_opacity(1 - t)  # t从0→1，透明度从1→0
            self.wait(magic_duration / steps)
        # ---------- 激活函数继续旋转一圈 ----------
        extra_rotations = 2
        extra_duration = 1.5
        extra_steps = 60
        for i in range(extra_steps):
            t = i / extra_steps
            angle_step = 2 * PI * extra_rotations / extra_steps * np.sin(t * np.pi)
            activation_text.rotate(angle_step, axis=UP)
            self.wait(extra_duration / extra_steps)